# Project 3: **Ask‑the‑Web Agent**

Welcome to Project 3! In this project, you will learn how to use tool‑calling LLMs, extend them with custom tools, and build a simplified *Perplexity‑style* agent that answers questions by searching the web.

## Learning Objectives  
* Understand why tool calling is useful and how LLMs can invoke external tools.
* Implement a minimal loop that parses the LLM's output and executes a Python function.
* See how *function schemas* (docstrings and type hints) let us scale to many tools.
* Use **LangChain** to get function‑calling capability
* Combine LLM with a web‑search tool to build a simple ask‑the‑web agent.
* Connect to external tools using **MCP (Model Context Protocol)**, a universal standard for LLM‑tool integration.
* Optionally build a UI using Chainlit to test your agent.

## Roadmap
0. Environment setup
1. Write simple tools and connect them to an LLM
2. Standardize tool calling with JSON schemas
3. Use LangGraph for tool calling
4. Build a Perplexity-style web-search agent
5. (Optional) MCP: connect to external tool servers
6. (Optional) A minimal UI

# 0- Environment setup

### Step 1: Create your environment and install dependencies 
Before we start coding, you need a reproducible setup. Open a terminal in the same directory as this notebook, and use Conda or uv to install the project dependencies.

#### Option 1: Conda


```bash
# Create and activate the conda environment
conda env create -f environment.yml && conda activate web_agent

```

#### Option 2: UV (faster)

If you prefer [uv](https://docs.astral.sh/uv/) over Conda:

```bash
# Install uv (skip if already installed)
curl -LsSf https://astral.sh/uv/install.sh | sh

# Create venv and install dependencies
uv venv .venv-web-agent-uv && source .venv-web-agent-uv/bin/activate
uv pip install -r requirements.txt
```

### Step 2: Register this environment as a Jupyter kernel
```bash
python -m ipykernel install --user --name=web_agent --display-name "web_agent"
```
Now open your notebook and switch to the `web_agent` kernel (Kernel → Change Kernel).

### Step 3: Set up Ollama

In this project, we use **Ollama** to load and use open-weight LLMs. We start with smaller models like `gemma3:1b` and then switch to larger models like `llama3.2:3b`.

Start the **Ollama** server in a terminal. This launches a local API endpoint that listens for LLM requests.

```bash
ollama serve
```

Downloads the model so you can run them locally without API calls. 
```bash
ollama pull gemma3:1b
ollama pull llama3.2:3b
```

You can explore other available models [here](https://ollama.com/library) and pull them to experiment with.

In [2]:
# Quick check: is Ollama running?
# If this fails, open a terminal and run: ollama serve

import httpx

response = httpx.get("http://localhost:11434/api/tags", timeout=5)
models = [m["name"] for m in response.json().get("models", [])]
print(f"Ollama is running. Installed models: {models}")

Ollama is running. Installed models: ['gemma3:1b', 'gemma3:12b', 'deepseek-r1:8b', 'llama3.2:3b', 'phi4-mini:latest', 'qwen2.5:3b-instruct', 'llama3.2:1b', 'deepseek-r1:latest', 'gemma3:270m', 'llama2:7b', 'llama3:latest', 'gemma3:latest']


## 1- Tool Calling

LLMs are strong at answering questions, but they cannot directly access external data such as live web results, APIs, or computations. In real applications, agents rarely rely only on their internal knowledge. They need to query APIs, retrieve data, or perform calculations to stay accurate and useful. Tool calling bridges this gap by allowing the LLM to request actions from the outside world.

<img src="assets/tools.png" width="700">

As show below, We first implement a tool, then describe the tool as part of the model's prompt. When the model decides that a tool is needed, it emits a structured output. A parser will detect this output, execute the corresponding function, and feed the result back to the LLM so the conversation continues.

<img src="assets/tool_flow.png" width="700">

In this section, you will implement a simple `get_current_weather` function and teach the `gemma3:1b` model to use it when required.

In [ ]:
# ---------------------------------------------------------
# Step 1: Implement the tool
# ---------------------------------------------------------
# You can either:
#   (a) Call a real weather API (for example, OpenWeatherMap), or
#   (b) Create a dummy function that returns a fixed response (e.g., "It is 23°C and sunny in San Francisco.")
#
# Output:
#   • Return a short, human-readable sentence describing the weather.
#
# Example expected behavior:
#   get_current_weather("San Francisco") → "It is 23°C and sunny in San Francisco."
#

def get_current_weather(city, unit = "celsius") -> str:
    return f"It is 23°{unit[0].upper()} and sunny in {city}." # no real API call to stay offline-friendly

In [ ]:
# ----------------------------------------------------------------------
# Step 2: Create a prompt to teach the LLM when and how to use your tool
# ----------------------------------------------------------------------
# What to include:
#   • A SYSTEM_PROMPT that tells the model about the tool use and describes the tool
#   • A USER_QUESTION with a user query that should trigger the tool.
#       Example: "What is the weather in San Diego today?"

SYSTEM_PROMPT = (
    "You are an assistant that can call tools. "
    "When the user asks something requiring fresh data, respond **only** with a JSON like: "
    'TOOL_CALL:{"name": <tool_name>, "args": { ... }}.'
)

TOOLS_SPEC = """
You can call exactly one tool:
- name: get_current_weather
  description: Return the current weather for a city.
  arguments:
    city: string
    unit: "celsius" | "fahrenheit"  (optional, default "celsius")
"""

# USER_QUESTION = "What is your name?"
USER_QUESTION = "What is the weather in San Diego today?"

Now that you have defined a tool and shown the model how to use it, the next step is to call the LLM using your prompt.

In [ ]:
# ---------------------------------------------------------
# Step 3: Call the LLM with your prompt
# ---------------------------------------------------------
# Task:
#   Send SYSTEM_PROMPT + USER_QUESTION to the model.
#
# Steps:
#   1. Create an Ollama client
#   2. Use chat.completions.create to send your prompt to gemma3:1b
#   3. Print the response.
#
# Expected:
#   The model should return something like:
#   TOOL_CALL: {"name": "get_current_weather", "args": {"city": "San Diego"}}
# ---------------------------------------------------------
from openai import OpenAI

client = OpenAI(api_key = "ollama", base_url = "http://localhost:11434/v1")

MODEL = "gemma3:1b"
# MODEL = "llama3.2:3b"

response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT + "\n\n" + TOOLS_SPEC},
        {"role": "user", "content": USER_QUESTION},
    ],
    temperature=0,
)

output = response.choices[0].message.content
print("\n Model output:\n", output)


 Model output:
 TOOL_CALL:{"name": "get_current_weather", "args": { "city": "San Diego", "unit": "celsius" }}



In [ ]:
# ---------------------------------------------------------
# Step 4: Manually parse the LLM output and call the tool
# ---------------------------------------------------------
# Task:
#   Detect when the model requests a tool, extract its name and arguments,
#   and execute the corresponding function.
#
# Steps:
#   1. Search for the text pattern "TOOL_CALL:{...}" in the model output.
#   2. Parse the JSON inside it to get the tool name and args.
#   3. Call the matching function (e.g., get_current_weather).
#
# Expected:
#   You should see a line like:
#       Calling tool `get_current_weather` with args {'city': 'San Diego'}
#       Result: It is 23°C and sunny in San Diego.
# ---------------------------------------------------------

import re, json

pattern = r'TOOL_CALL:\s*({.*})'
match = re.search(pattern, output, flags=re.DOTALL)
if match:
    call_json = json.loads(match.group(1))
    fn_name   = call_json["name"]
    args      = call_json.get("args", {})
    print(f"\nCalling tool `{fn_name}` with args {args}")
    result = globals()[fn_name](**args)
    print("Result:", result)
else:
    print("No tool call detected")


Calling tool `get_current_weather` with args {'city': 'San Diego', 'unit': 'celsius'}
Result: It is 23°C and sunny in San Diego.


# 2- Standardize tool calling

So far, we handled tool calling manually by writing a function, manually teaching the LLM about it, and write a regex to parse the output. This approach does not scale if we want to add more tools. Adding more tools would mean more `if/else` blocks and manual edits to the prompt.

To make the system flexible, we can standardize tool definitions by automatically reading each function's signature, converting it to a JSON schema, and passing that schema to the LLM. This way, the LLM can dynamically understand which tools exist and how to call them without requiring manual updates to prompts or conditional logic.

Next, you will implement a small helper that extracts metadata from functions and builds a schema for each tool.

In [ ]:
# ---------------------------------------------------------
# Generate a JSON schema for a tool automatically
# ---------------------------------------------------------
#
# Steps:
#   1. Rewrite the get_current_weather function with docstring and arg types
#   2. Use `inspect.signature` to automatically get function parameters and docstring
#   2. For each argument, record its name, type, and description.
#   3. Build a schema containing: name, description, and parameters.
#   4. Test your helper on `get_current_weather` and print the result.
#
# Expected:
#   A dictionary describing the tool (its name, args, and types).
# ---------------------------------------------------------

from pprint import pprint
import inspect

def get_current_weather(city: str, unit: str = "celsius") -> str:
    """Return the current weather as a human-readable sentence."""
    return f"It is 23°{unit[0].upper()} and sunny in {city}." # no real API call to stay offline-friendly

def to_schema(fn):
    sig = inspect.signature(fn)
    props = {
        name: {
            'type': 'string' if param.annotation is str else 'number',
            'description': f"Argument {name}"
        }
        for name, param in sig.parameters.items()
    }
    return {
        'name': fn.__name__,
        'description': (fn.__doc__ or '').strip().split('\n')[0],
        'parameters': {
            'properties': props,
            'required': [n for n, p in sig.parameters.items() if p.default is inspect._empty]
        }
    }

tool_schema = to_schema(get_current_weather)
pprint(tool_schema)

{'description': 'Return the current weather as a human-readable sentence.',
 'name': 'get_current_weather',
 'parameters': {'properties': {'city': {'description': 'Argument city',
                                        'type': 'string'},
                               'unit': {'description': 'Argument unit',
                                        'type': 'string'}},
                'required': ['city']}}


In [9]:
# ---------------------------------------------------------
# Provide the tool schema to the model instead of prompt surgery
# ---------------------------------------------------------
# Goal:
#   Give the model a "menu" of available tools so it can choose
#   which one to call based on the user’s question.
#
# Steps:
#   1. Add an extra system message (e.g., name="tool_spec")
#      containing the JSON schema(s) of your tools.
#   2. Include SYSTEM_PROMPT and the user question as before.
#   3. Send the messages to the model (e.g., gemma3:1b).
#   4. Print the model output to see if it picks the right tool.
#
# Expected:
#   The model should produce a structured TOOL_CALL indicating
#   which tool to use and with what arguments.
# ---------------------------------------------------------

messages=[
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "system", "name": "tool_spec", "content": json.dumps([tool_schema])},
    {"role": "user", "content": "Is Boston warmer today or Seattle?"}
]
MODEL = "gemma3:1b" # doesn't produce two calls
# MODEL = "llama3.2:3b" # produces two calls
output = client.chat.completions.create(
    model=MODEL, 
    messages=messages,
    temperature=0.5,
)
print(output.choices[0].message.content)

TOOL_CALL:{"name": "get_current_weather", "args": { "properties": { "city": "Boston" }, "unit": "Celsius"} }


## 3- LangChain for Tool Calling

So far, you built a simple tool-calling pipeline. While this helps you understand the logic, it does not scale well when working with multiple tools, complex parsing, or multi-step reasoning. We have to write manual parsers, function calling logic, and adding responses back to the prompt.

LangChain simplifies this process. You only need to declare your tools, and its *Agent* abstraction handles when to call a tool, how to use it, and how to continue reasoning afterward. In this section, you will create a **ReAct** Agent (Reasoning + Acting). As shown below, the model alternates between reasoning steps and tool use wihtout any manual work.

<img src="assets/react.png" width="500">

The following links might be helpful for completing this section:
- [Create Agents](https://docs.langchain.com/oss/python/langchain/agents)
- [LangChain Tools](https://docs.langchain.com/oss/python/langchain/tools)
- [Ollama](https://docs.langchain.com/oss/python/integrations/chat/ollama)

In [10]:
# ---------------------------------------------------------
# Step 1: Define tools for LangChain
# ---------------------------------------------------------
# Steps:
#   1. Keep your existing `get_current_weather` function as before.
#   2. Create a new function (e.g., get_weather) that calls it.
#   3. Add the `@tool` decorator so LangChain can register it automatically.
#
# Notes:
#   • The decorator converts your Python function into a standardized tool object.

from langchain_core.tools import tool

def get_current_weather(city: str, unit: str = "celsius") -> str:
    """Return the current weather as a human-readable sentence."""
    return f"It is 23°{unit[0].upper()} and sunny in {city}."

@tool
def get_weather(city: str) -> str:
    """Get current weather for a city and return results."""
    return get_current_weather(city)

In [11]:
# ---------------------------------------------------------
# Step 2: Create the Agent
# ---------------------------------------------------------
# Steps:
#   1. Create a gemma3:1b LLM instance 
#   2. Create the agent using create_agent
#   3. Test the agent with a natural question using agent.invoke

from langchain_ollama import ChatOllama
from langchain.agents import create_agent

# Create an LLM instance
MODEL = "gemma3:1b"
llm = ChatOllama(model=MODEL, temperature=0)

# Create your tool list
tools = [get_weather]

# Create your agent
agent = create_agent(llm, tools)

# Call your agent using agent.invoke
messages={"messages":
    [{"role": "user", "content": "Do I need an umbrella in Seattle today?"}]
}
out = agent.invoke(messages)
print(out["messages"][-1].content)

ResponseError: registry.ollama.ai/library/gemma3:1b does not support tools (status code: 400)

### What just happened?
Your run failed because `gemma3:1b` does not support native tool calling (function calling). LangChain expects the model to return a structured tool-call object, but `gemma3:1b` can only return plain text, so the tool invocation step breaks.

### Why previosuly, our manual approach worked with any model?

In previous sections, we used **text-based tool calling**. We described the tool format in the system prompt. We asked the model to output `TOOL_CALL: {"name": ..., "args": ...}`. We then parsed this text with regex.

This works with **any model** (even small ones like `gemma3:1b`) because we're just asking the model to follow a certain structured output format.

### Why LangChain requires specific models?

LangChain relies on **native tool calling** and it expects a consistent structured output format irrespective of the model. Hence, it enfornces model outputs structured tool calls in a specific format. This requires models trained specifically for function calling

**Rule of thumb**: Models under 3B parameters typically lack native tool-calling capability.

| Model | Size | Native Tool Support | Notes |
|-------|------|---------------------|-------|
| `gemma3:1b` | 1B | No | Works for manual approach only |
| `llama3.2:1b` | 1B | No | Works for manual approach only |
| `llama3.2:3b` | 3B | Yes | Good balance of speed and capability |
| `gemma3` | 4B | Yes | Supports native tools |
| `mistral` | 7B | Yes | Strong tool support |

Let's fix the issue we observed in the previous cell.



In [12]:
# ---------------------------------------------------------
# Step 2 (retry): Re-create the Agent with a native tool-calling LLM
# ---------------------------------------------------------
# Steps:
#   1. Create a llama3.2:3b LLM instance 
#   2. Create a system prompt to teach react-style reasoning to the LLM
#   3. Create the agent using create_agent
#   4. Test the agent with a natural question using agent.invoke

from langchain_ollama import ChatOllama
from langchain.agents import create_agent

# Create an LLM instance
MODEL = "llama3.2:3b"
llm = ChatOllama(model=MODEL, temperature=0)

# Create your tool list
tools = [get_weather]

# Create a ReAct-Style System Prompt
react_system_prompt = """
You are a ReAct-style tool-using assistant.

Loop:
- Decide if you need a tool.
- If yes, call one tool with a specific input.
- Read the result and repeat until you can answer.

Rules:
- Use tools for factual, time-sensitive, or “latest/current/price/schedule/who is” questions.
- Never invent tool outputs or sources.
- If tools fail or are insufficient, say what you could not verify.

Output:
- Give a clear final answer. Do not reveal internal reasoning.
"""

# Create your agent
agent = create_agent(llm, tools, system_prompt=react_system_prompt)

# Call your agent using agent.invoke
messages={"messages":
    [{"role": "user", "content": "Do I need an umbrella in Seattle today?"}]
}
out = agent.invoke(messages)
print(out["messages"][-1].content)

You don't need an umbrella in Seattle today.


## 4- Web Search Agent

Now that you know how to use LangChain with tools, let's build something useful. Instead of a toy get_weather tool, let create an agent that searches the web and answers questions using real results. In the next section, you will create a [DuckDuckGo](https://github.com/deedy5/ddgs) search tool and wire it into a ReAct agent.

In [13]:
# ---------------------------------------------------------
# Step 1: Write a web search tool
# ---------------------------------------------------------
# Steps:
#   1. Write a helper function (e.g., search_web) that:
#        • Takes a query string
#        • Uses DuckDuckGo (DDGS) to fetch top results (titles + URLs)
#        • Returns them as a formatted string
#   2. Wrap it with the @tool decorator to make it available to LangChain.


from ddgs import DDGS
from langchain_core.tools import tool

def search_web(query: str, max_results: int = 10) -> str:
    """Return the top `max_results` web results for `query` as a single
    formatted string. Uses DuckDuckGo's unofficial API."""
    results = []
    with DDGS() as ddgs:
        for r in ddgs.text(query, max_results=max_results):
            results.append(f"- {r['title']} — {r['href']}")
    return "\n".join(results)

@tool
def web_search(query: str) -> str:
    """Search the web for a query and return concise results."""
    return search_web(query)

In [14]:
# ---------------------------------------------------------
# Step 2: Initialize the web-search agent
# ---------------------------------------------------------
# Steps:
#   1. Create an LLM (e.g., ChatOllama).
#   2. Add your `web_search` tool to the tools list.
#   3. Create the agent using create_agent.
#
# Expected:
#   The agent should be ready to accept user queries
#   and use your web search tool when needed.
# ---------------------------------------------------------

from langchain_ollama import ChatOllama
from langchain.agents import create_agent

MODEL = "llama3.2:3b"
llm = ChatOllama(model=MODEL, temperature=0)
tools = [web_search]

web_agent = create_agent(llm, tools)

def run_agent_with_reasoning(agent, question: str) -> str:
    """
    Run the agent and display reasoning steps.
    Returns the final answer.
    """
    print("=" * 60)
    print(f"Question: {question}")
    print("=" * 60)
    
    final_answer = ""
    
    for step in agent.stream({"messages": [("user", question)]}):
        for node_name, node_output in step.items():
            print(f"\n[{node_name.upper()}]")
            
            if "messages" in node_output:
                for msg in node_output["messages"]:
                    if hasattr(msg, "tool_calls") and msg.tool_calls:
                        for tc in msg.tool_calls:
                            print(f"  Tool: {tc['name']}")
                            print(f"  Args: {tc['args']}")
                    elif hasattr(msg, "content") and msg.content:
                        content = str(msg.content)
                        # Truncate long outputs (like search results)
                        if len(content) > 500:
                            print(f"  {content[:500]}...")
                        else:
                            print(f"  {content}")
                        final_answer = content
    
    print("\n" + "=" * 60)
    return final_answer

In [15]:
# ---------------------------------------------------------
# Step 3: Test your Ask-the-Web agent using agent.invoke
# ---------------------------------------------------------
try:
    answer = run_agent_with_reasoning(
        web_agent, 
        "What are the current events in San Francisco this week?"
    )
except Exception as e:
    print(f"Error: {e}")

Question: What are the current events in San Francisco this week?

[MODEL]
  Tool: web_search
  Args: {'query': 'San Francisco current events this week'}

[TOOLS]
  - San Francisco - Wikipedia — https://en.wikipedia.org/wiki/San_Francisco
- ISACA San Francisco Current Events — http://www.sfisaca.org/events.htm
- Current Local Time in San Francisco , California, USA — https://www.timeanddate.com/worldclock/usa/san-francisco
- San Francisco Raids Ice | TikTok — https://www.tiktok.com/discover/san-francisco-raids-ice
- Time in San Francisco , California, United States now — https://time.is/San_Francisco
- San Francisco Current Events - YouTube — https://www.yo...

[MODEL]
  It appears that there are several current events happening in San Francisco this week. Here are a few:

* Riordan High School has suspended in-person classes for one week due to a tuberculosis outbreak.
* The San Francisco Giants are playing at home, and fans can catch their games on the official MLB.com website.
* The

## 5- (Optional) MCP: Model Context Protocol

Up to now, every tool you used started as a Python function you wrote and registered yourself. **MCP (Model Context Protocol)** lets you skip that step. Tools come from an external *server*, and your code just connects to it. Think of it like USB for AI tools: any MCP client can plug into any MCP server and immediately use whatever tools it offers.

Below, we connect to `mcp-server-fetch` (a ready-made server that can retrieve any URL) using the Python MCP SDK. We launch the server, discover its tools, and call one, all without writing a single `@tool` function. To learn more, read: https://github.com/modelcontextprotocol/servers/tree/main/src/fetch

> **LangChain integration:** The `langchain-mcp-adapters` package can convert MCP tools into LangChain-compatible tools automatically, so you can drop them straight into a ReAct agent like the ones in section 4.

In [16]:
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

# Two MCP servers — different capabilities, same protocol
fetch_params = StdioServerParameters(command="uvx", args=["mcp-server-fetch"])
fs_params = StdioServerParameters(
    command="npx", args=["-y", "@modelcontextprotocol/server-filesystem", "/tmp"],
)

for label, params in [("fetch", fetch_params), ("filesystem", fs_params)]:
    async with stdio_client(params) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()
            result = await session.list_tools()
            print(f"\n{label} server — {len(result.tools)} tool(s):")
            for t in result.tools:
                print(f"  {t.name}: {t.description[:80]}")


fetch server — 1 tool(s):
  fetch: Fetches a URL from the internet and optionally extracts its contents as markdown

filesystem server — 14 tool(s):
  read_file: Read the complete contents of a file as text. DEPRECATED: Use read_text_file ins
  read_text_file: Read the complete contents of a file from the file system as text. Handles vario
  read_media_file: Read an image or audio file. Returns the base64 encoded data and MIME type. Only
  read_multiple_files: Read the contents of multiple files simultaneously. This is more efficient than 
  write_file: Create a new file or completely overwrite an existing file with new content. Use
  edit_file: Make line-based edits to a text file. Each edit replaces exact line sequences wi
  create_directory: Create a new directory or ensure a directory exists. Can create multiple nested 
  list_directory: Get a detailed listing of all files and directories in a specified path. Results
  list_directory_with_sizes: Get a detailed listing of all files

In [ ]:
from langchain_mcp_adapters.tools import load_mcp_tools

from langchain.agents import create_agent
from langchain_ollama import ChatOllama

async with stdio_client(fetch_params) as (read, write):
    async with ClientSession(read, write) as session:
        await session.initialize()

        # In section 4: tools = [web_search]
        # With MCP:     tools come from the server
        tools = await load_mcp_tools(session)

        # Small models sometimes pass null for optional int params.
        # Strip None values so the server's JSON-schema validation won't reject them.
        for t in tools:
            _orig = t.coroutine
            async def _strip_none(_f=_orig, **kw):
                return await _f(**{k: v for k, v in kw.items() if v is not None})
            t.coroutine = _strip_none

        # From here, same as previous section
        llm = ChatOllama(model="llama3.2:3b", temperature=0)
        agent = create_agent(llm, tools)

        # ainvoke (not invoke) because MCP tools are async
        out = await agent.ainvoke(
            {"messages": [("user", "Fetch https://python.org and summarize the page")]}
        )
        print(out["messages"][-1].content)

The Python website provides various resources for users, including:

* Download links for the latest version of Python (Python 3.14.2)
* Documentation for the standard library, tutorials, and guides
* A community-run job board for finding work or hiring staff with Python-related skills
* News and updates about the Python project, including announcements and new features

Additionally, the website lists various applications and tools that can be used with Python, including:

* Web development frameworks such as Django, Pyramid, Bottle, Tornado, Flask, Litestar, and web2py
* GUI development libraries like tkInter, PyGObject, PyQt, PySide, Kivy, wxPython, and DearPyGui
* Scientific and numeric libraries like SciPy, Pandas, and IPython
* Software development tools like Buildbot, Trac, and Roundup
* System administration tools like Ansible, Salt, OpenStack, and xonsh

Overall, the Python website is a comprehensive resource for users looking to learn more about the language, its applications

In [18]:
# You can also call MCP tools directly, without LangChain
async with stdio_client(fetch_params) as (read, write):
    async with ClientSession(read, write) as session:
        await session.initialize()

        result = await session.call_tool("fetch", arguments={"url": "https://python.org"})
        for block in result.content:
            if hasattr(block, "text"):
                print(block.text)

Contents of https://python.org/:
**Notice:** This page displays a fallback because interactive scripts did not run. Possible causes include disabled JavaScript or failure to load scripts or stylesheets.

## Download

Python source code and installers are available for download for all versions!

Latest: [Python 3.14.2](/downloads/release/python-3142/)

## Docs

Documentation for Python's standard library, along with tutorials and guides, are available online.

[docs.python.org](https://docs.python.org/)

## Jobs

Looking for work or have a Python related position that you're trying to hire for? Our **relaunched community-run job board** is the place to go.

[jobs.python.org](//jobs.python.org)

## Latest News

[More](https://blog.python.org/ "More News")

* 2026-01-26
  [Your Python. Your Voice. Join the Python Developers Survey 2026!](https://pyfound.blogspot.com/2026/01/your-python-your-voice-join-python.html)
* 2026-01-21
  [Departing the Python Software Foundation (Staff)](https://

## 6- (Optional) A Minimal UI

[Chainlit](https://chainlit.io/) is a Python library designed specifically for building LLM and agent UIs. It provides:
- Built-in streaming support
- Message history
- Step visualization (see tool calls as they happen)
- No frontend code required

If you are interested, follow Chainlit's documentation to implement a simple UI for your agent. The process typically involves:

1. You write a Python file named `chainlit_app.py` with the agent creation logic as well as UI handlers (e.g.,`@cl.on_message`)
2. Run the file in your terminal with `chainlit run app.py`
3. A web UI opens automatically at `http://localhost:8000`

In [100]:
%%writefile chainlit_app.py
# ---------------------------------------------------------
# Chainlit Web Search Agent

import chainlit as cl
from langchain_ollama import ChatOllama
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool
from langchain_core.messages import AIMessage, ToolMessage
from ddgs import DDGS


# ---------------------------------------------------------
# Define the web search tool
# ---------------------------------------------------------
@tool
def web_search(query: str) -> str:
    """Search the web for a query and return results."""
    results = []
    with DDGS() as ddgs:
        for r in ddgs.text(query, max_results=5):
            results.append(f"- {r['title']}: {r['href']}")
    return "\n".join(results) if results else "No results found."


# ---------------------------------------------------------
# Create the agent (once at startup)
# ---------------------------------------------------------
MODEL = "llama3.2:3b"
llm = ChatOllama(model=MODEL, temperature=0)
agent = create_react_agent(llm, [web_search])


# ---------------------------------------------------------
# Chainlit message handler
# ---------------------------------------------------------
@cl.on_message
async def handle_message(message: cl.Message):
    """Handle user messages and stream agent responses."""
    
    final_answer = ""
    
    # Stream through agent steps
    for step in agent.stream({"messages": [("user", message.content)]}):
        for node_name, node_output in step.items():
            if "messages" not in node_output:
                continue
                
            for msg in node_output["messages"]:
                # AIMessage: either a tool call or the final answer
                if isinstance(msg, AIMessage):
                    if msg.tool_calls:
                        # Agent decided to call a tool
                        for tc in msg.tool_calls:
                            async with cl.Step(name=f"🔍 {tc['name']}") as step:
                                step.output = f"Query: {tc['args'].get('query', tc['args'])}"
                    elif msg.content:
                        # Final answer from the agent
                        final_answer = msg.content
                
                # ToolMessage: results from a tool call
                elif isinstance(msg, ToolMessage):
                    async with cl.Step(name="📄 Search Results") as step:
                        content = msg.content
                        step.output = content[:1000] + "..." if len(content) > 1000 else content
    
    # Send the final answer
    if final_answer:
        await cl.Message(content=final_answer).send()
    else:
        await cl.Message(content="I couldn't generate a response. Please try again.").send()


# ---------------------------------------------------------
# Welcome message
# ---------------------------------------------------------
@cl.on_chat_start
async def start():
    await cl.Message(
        content="👋 Welcome! I'm an Ask-the-Web agent. Ask me anything and I'll search the web to find answers.\n\n"
                "Try: **What are the current events in San Francisco this week?**"
    ).send()

Overwriting chainlit_app.py


## 🎉 Congratulations!

You have built a **web-enabled agent** from scratch: manual tool calling → JSON schemas → LangChain ReAct → web search → MCP → UI.

Next steps:
* Try adding more tools, such as news or finance APIs.
* Experiment with multiple tools, different models, and measure accuracy vs. hallucination.
* Explore the [MCP server registry](https://github.com/modelcontextprotocol/servers) for ready-made tool servers.

👏 **Great job!** Take a moment to celebrate. The techniques you implemented here power many production agents and chatbots.